In [1]:
import schedule, time, datetime
from ftx.client import FtxClient
from arbitrage import open_arbitrage, close_arbitrage
from settings import ARBITRAGE_C, ARBITRAGE_NC

# Target and batch size setup

In [2]:
COINS = {
    ARBITRAGE_C['SUBACCOUNT']: {
        'target': 'RUNE-PERP',
        'batch': 3,
    },
    ARBITRAGE_NC['SUBACCOUNT']: {
        'target': 'TRU-PERP',
        'batch': 20,
    },
}

In [3]:
collateral_arbitrageClient = FtxClient(api_key=ARBITRAGE_C['API'],
                   api_secret=ARBITRAGE_C['SECRET'],
                   subaccount_name=ARBITRAGE_C['SUBACCOUNT'])
nonCollateral_arbitrageClient = FtxClient(api_key=ARBITRAGE_NC['API'],
                   api_secret=ARBITRAGE_NC['SECRET'],
                   subaccount_name=ARBITRAGE_NC['SUBACCOUNT'])

# Loading Functions

In [4]:
def auto_arbitrage(client: FtxClient, coin: dict) -> None:
    print('[auto_arbitrage] DEBUG: <{}> {}'.format(coin['target'], datetime.datetime.now()))
    usd_balance = client.get_balance_coin('USD').get('total', 0)
    account_info = client.get_account_info()
    leverage = 1/account_info['marginFraction']
    future = client.get_single_market(coin['target'])
    spot = client.get_single_market(coin['target'].split('-')[0] + '/USD')
    if leverage < 3 and usd_balance >= 30:
        spread = future.get('bid', 0) / spot.get('ask', 0)
        if spread > 1.002:
            print('[auto_arbitrage] DEBUG: <{}> Open {} positions.'.format(coin['target'], coin['batch']))
            open_arbitrage(client, coin['target'], coin['batch'], coin['batch'])
        else:
            print('[auto_arbitrage] DEBUG: <{}> Spread = {}. Do nothing.'.format(coin['target'], spread))
    elif leverage > 4:
        print('[auto_arbitrage] DEBUG: <{}> Close {} positions.'.format(coin['target'], coin['batch']))
        close_arbitrage(client, coin['target'], coin['batch'], coin['batch'])
    else:
        print('[auto_arbitrage] DEBUG: <{}> Leverage = {}. Do nothing.'.format(coin['target'], leverage))        

In [5]:
def stop_arbitrage(client: FtxClient, coin: dict) -> None:
    print('[stop_arbitrage] DEBUG: <{}> {}'.format(coin['target'], datetime.datetime.now()))
    future_balance = client.get_position(coin['target']).get('size', 0)
    spot_balance = client.get_balance_coin(coin['target'].split('-')[0]).get('total', 0)
    if spot_balance >= coin['batch'] and future_balance >= coin['batch']:
        future = client.get_single_market(coin['target'])
        spot = client.get_single_market(coin['target'].split('-')[0] + '/USD')
        spread = future.get('ask', 0) / spot.get('bid', 0)
        if spread < 0.998:
            print('[stop_arbitrage] DEBUG: <{}> Close {} positions.'.format(coin['target'], coin['batch']))
            close_arbitrage(client, coin['target'], coin['batch'], coin['batch'])
        else:
            print('[stop_arbitrage] DEBUG: <{}> Spread = {}. Do nothing.'.format(coin['target'], spread))
    else:
        if spot_balance > 0 or future_balance > 0:
            print('[stop_arbitrage] DEBUG: <{}> Close {} positions.'.format(coin['target'], future_balance))
            close_arbitrage(client, coin['target'], spot_balance, future_balance)
        else:
            print('[stop_arbitrage] DEBUG: <{}> Balances = {}, {}. Do nothing.'.format(coin['target'], spot_balance, future_balance))

In [6]:
def main() -> None:
    schedule.every(5).seconds.do(auto_arbitrage, collateral_arbitrageClient, COINS[ARBITRAGE_C['SUBACCOUNT']])
    schedule.every(5).seconds.do(auto_arbitrage, nonCollateral_arbitrageClient, COINS[ARBITRAGE_NC['SUBACCOUNT']])
    schedule.every(5).seconds.do(stop_arbitrage, collateral_arbitrageClient, COINS[ARBITRAGE_C['SUBACCOUNT']])
    schedule.every(5).seconds.do(stop_arbitrage, nonCollateral_arbitrageClient, COINS[ARBITRAGE_NC['SUBACCOUNT']])
    while True:
        schedule.run_pending()
        time.sleep(1)

# Start the Spot-Future Arbitrage Schedule

In [7]:
if __name__ == '__main__':
    main()

[auto_arbitrage] DEBUG: <RUNE-PERP> 2021-10-03 22:42:34.232871
[auto_arbitrage] DEBUG: <RUNE-PERP> Leverage = 1.4961678403629197. Do nothing.
[auto_arbitrage] DEBUG: <TRU-PERP> 2021-10-03 22:42:34.932911
[auto_arbitrage] DEBUG: <TRU-PERP> Leverage = 3.469038273756042. Do nothing.
[stop_arbitrage] DEBUG: <RUNE-PERP> 2021-10-03 22:42:35.624951
[stop_arbitrage] DEBUG: <RUNE-PERP> Spread = 1.001244766323413. Do nothing.
[stop_arbitrage] DEBUG: <TRU-PERP> 2021-10-03 22:42:36.265987
[stop_arbitrage] DEBUG: <TRU-PERP> Spread = 1.0032. Do nothing.


KeyboardInterrupt: 

# Close the Spot-Future Arbitrage

## Collateral Arbitrage Account

In [ ]:
stop_arbitrage(collateral_arbitrageClient, COINS[ARBITRAGE_C['SUBACCOUNT']])

## Non-collateral Arbitrage Account

In [ ]:
stop_arbitrage(nonCollateral_arbitrageClient, COINS[ARBITRAGE_NC['SUBACCOUNT']])